In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PipelinePromptTemplate
from langchain.prompts import PromptTemplate, ChatPromptTemplate


chat = ChatOpenAI(temperature=0.9,streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a specialist for writing Haikus about programming languages."),
    ("human","Plaese write Haiku about {language}.")
])

critic_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in explaining Haikus."),
    ("human","Plaese explaining about{poem}.")
])


poet_chain = poet_prompt | chat 
critic_chain = critic_prompt | chat
final_chain = {"poem": poet_chain} | critic_chain
final_chain.invoke({"language":"Java"})


Java, strong and vast,
Object-oriented bliss,
Code runs everywhere.A haiku is a form of Japanese poetry consisting of three lines with a syllable pattern of 5-7-5. Haikus often focus on nature, seasons, or emotions, and aim to capture a moment or feeling in a concise and evocative way. The brevity of haikus encourages readers to pause and reflect on the imagery and emotions conveyed in just a few words. Traditional haikus also include a kigo, or seasonal reference, which helps to set the scene and create a sense of atmosphere. Overall, haikus are a beautiful and contemplative form of poetry that can evoke a sense of mindfulness and connection to the natural world.

AIMessageChunk(content='A haiku is a form of Japanese poetry consisting of three lines with a syllable pattern of 5-7-5. Haikus often focus on nature, seasons, or emotions, and aim to capture a moment or feeling in a concise and evocative way. The brevity of haikus encourages readers to pause and reflect on the imagery and emotions conveyed in just a few words. Traditional haikus also include a kigo, or seasonal reference, which helps to set the scene and create a sense of atmosphere. Overall, haikus are a beautiful and contemplative form of poetry that can evoke a sense of mindfulness and connection to the natural world.')